In [ ]:
!pip install wrds stockstats alpaca_trade_api stable_baselines3 gymnasium finrl pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade scipy==1.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.12.0
    Uninstalling scipy-1.12.0:
      Successfully uninstalled scipy-1.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wrds 3.2.0 requires scipy<1.13,>=1.12, but you have scipy 1.13.0 which is incompatible.


In [ ]:
path = '/content/sample_data'

In [ ]:
# Standard library imports
import sys, itertools, os, re
from datetime import datetime, timedelta

# Third-party library imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas_market_calendars as mcal

# Local application/library specific imports
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

TRAIN_START_DATE = '2015-01-01'
TRAIN_END_DATE = '2024-09-19'
TRADE_START_DATE = '2024-09-19'
TRADE_END_DATE = '2024-10-01'

subsample = [150,200]
train_path = f'/content/sample_data/train_data_{subsample[0]}_{subsample[1]}.csv'
trade_path = f'/content/sample_data/trade_data_{subsample[0]}_{subsample[1]}.csv'
raw_path = f'/content/sample_data/raw_data_{subsample[0]}_{subsample[1]}.csv'

In [ ]:
# from config import *

bbg_data = pd.read_excel('/content/sample_data/bbg_draft_final.xlsx', sheet_name='Sheet1')
symbols = list(bbg_data['Symbol'].apply(lambda x: re.split(' |/', x)[0]))
sentiment = pd.read_excel('/content/sample_data/bbg_draft_final.xlsx', sheet_name='Sheet2')

tickers = symbols[subsample[0]: subsample[1]]

df_raw = YahooDownloader(start_date = TRAIN_START_DATE, end_date = TRADE_END_DATE, ticker_list = tickers).fetch_data()

df_raw.to_csv(raw_path)

fe = FeatureEngineer(use_technical_indicator=True,
            tech_indicator_list = INDICATORS,
            use_vix=True,
            use_turbulence=True,
            user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
# trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)

# df = pd.DataFrame(0, index=range(len(set(trade['tic'].tolist()))), columns=[f'Column_{i}' for i in range(1, trade.shape[1]+1)])
# df.columns = trade.columns

# train.to_csv(train_path)
# df.to_csv(trade_path)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (103572, 8)


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (2451, 8)
Successfully added vix


Successfully added turbulence index


In [ ]:
def merge_sentiment(df):
  date = str(df['date'])[:10]
  ticker = str(df['tic'])
  try:
    sent_date = sentiment[ticker+'  US Equity']
    sent_date = sent_date.apply(lambda x: str(x)[:10])
    for j in range(len(sent_date)):
      if sent_date[j] == date:
        break

    index = j
    i = symbols.index(ticker)
    col = 'Unnamed: '+str(2*i+1)
    return sentiment.loc[index, col]
  except:
    return np.nan

In [ ]:
subsample = [40, 100]
trade_path = train_path = f'/content/sample_data/train_data_{subsample[0]}_{subsample[1]}.csv'
train = pd.read_csv(train_path, index_col='Unnamed: 0')
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
0,2015-01-02,ACNB,20.900000,21.900000,20.900000,15.550253,2300.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,15.550253,15.550253,17.790001,0.000000,0.0000
0,2015-01-02,ALCO,50.000000,50.029999,49.220001,42.621014,21200.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,42.621014,42.621014,17.790001,0.000000,0.0000
0,2015-01-02,ATLO,26.059999,26.059999,25.290001,17.146029,9100.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,17.146029,17.146029,17.790001,0.000000,0.0000
0,2015-01-02,BCML,11.900000,11.900000,11.900000,11.374499,0.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,11.374499,11.374499,17.790001,0.000000,0.0000
0,2015-01-02,BFC,22.650000,22.650000,22.650000,19.334398,0.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,19.334398,19.334398,17.790001,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,2024-09-18,TWIN,12.600000,12.670000,12.290000,12.290000,17800.0,2.0,-0.127705,13.419745,11.622255,47.037658,-12.439415,12.788529,12.452065,12.737876,18.230000,33.865259,0.1068
2443,2024-09-18,UNTY,33.470001,34.570000,33.439999,34.029999,27700.0,2.0,0.306893,34.297773,31.507707,59.062543,124.131115,13.449712,32.389266,31.784323,18.230000,33.865259,0.1068
2443,2024-09-18,UTMD,67.309998,68.660004,66.900002,66.923447,23500.0,2.0,-0.239490,68.621248,65.188800,48.652010,33.719258,10.314320,66.861716,67.662832,18.230000,33.865259,0.1068
2443,2024-09-18,VLGEA,32.889999,33.200001,32.540001,32.278343,27600.0,2.0,0.854012,33.008502,30.254666,60.419116,87.125076,24.515796,30.461073,28.881709,18.230000,33.865259,0.1068


In [ ]:
train['sentiment'] = train.apply(merge_sentiment, axis=1)
# train['sentiment'] = train['sentiment'].ffill().fillna(0)
train.to_csv(train_path)
train[train['sentiment']!=0]

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
0,2015-01-02,ACNB,20.900000,21.900000,20.900000,15.550253,2300.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,15.550253,15.550253,17.790001,0.000000,NaN
0,2015-01-02,ALCO,50.000000,50.029999,49.220001,42.621014,21200.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,42.621014,42.621014,17.790001,0.000000,NaN
0,2015-01-02,ATLO,26.059999,26.059999,25.290001,17.146029,9100.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,17.146029,17.146029,17.790001,0.000000,NaN
0,2015-01-02,BCML,11.900000,11.900000,11.900000,11.374499,0.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,11.374499,11.374499,17.790001,0.000000,NaN
0,2015-01-02,BFC,22.650000,22.650000,22.650000,19.334398,0.0,4.0,0.000000,15.843236,14.936794,0.000000,-66.666667,100.000000,19.334398,19.334398,17.790001,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,2024-09-18,TWIN,12.600000,12.670000,12.290000,12.290000,17800.0,2.0,-0.127705,13.419745,11.622255,47.037658,-12.439415,12.788529,12.452065,12.737876,18.230000,33.865259,NaN
2443,2024-09-18,UNTY,33.470001,34.570000,33.439999,34.029999,27700.0,2.0,0.306893,34.297773,31.507707,59.062543,124.131115,13.449712,32.389266,31.784323,18.230000,33.865259,NaN
2443,2024-09-18,UTMD,67.309998,68.660004,66.900002,66.923447,23500.0,2.0,-0.239490,68.621248,65.188800,48.652010,33.719258,10.314320,66.861716,67.662832,18.230000,33.865259,NaN
2443,2024-09-18,VLGEA,32.889999,33.200001,32.540001,32.278343,27600.0,2.0,0.854012,33.008502,30.254666,60.419116,87.125076,24.515796,30.461073,28.881709,18.230000,33.865259,NaN


In [ ]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
0,2015-01-02,AROW,23.980440,23.980440,23.414248,16.808167,20205.0,4.0,0.000000,16.836607,16.748618,0.000000,-66.666667,100.000000,16.808167,16.808167,17.790001,0.000000,NaN
0,2015-01-02,BHB,21.280001,21.280001,20.680000,15.044217,2100.0,4.0,0.000000,16.836607,16.748618,0.000000,-66.666667,100.000000,15.044217,15.044217,17.790001,0.000000,NaN
0,2015-01-02,BSRR,17.620001,17.639999,17.000000,12.533772,4700.0,4.0,0.000000,16.836607,16.748618,0.000000,-66.666667,100.000000,12.533772,12.533772,17.790001,0.000000,NaN
0,2015-01-02,CASS,40.659092,40.659092,37.780304,30.696018,29964.0,4.0,0.000000,16.836607,16.748618,0.000000,-66.666667,100.000000,30.696018,30.696018,17.790001,0.000000,NaN
0,2015-01-02,CBAN,7.880000,7.880000,7.880000,6.496354,0.0,4.0,0.000000,16.836607,16.748618,0.000000,-66.666667,100.000000,6.496354,6.496354,17.790001,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,2024-09-18,WINA,370.609985,382.500000,369.190002,373.029999,25300.0,2.0,-1.171311,378.221365,341.913639,52.571028,120.428737,18.042667,361.872278,369.062660,18.230000,18.124113,NaN
2443,2024-09-18,WLFC,119.930000,127.599998,119.930000,123.239998,24400.0,2.0,6.632292,123.522494,95.715505,70.996754,178.365258,55.182848,105.277616,91.278646,18.230000,18.124113,NaN
2443,2024-09-18,WRLD,125.180000,129.270004,123.360001,125.519997,45100.0,2.0,0.962527,125.673252,107.536749,54.098170,175.764387,10.467976,113.965666,120.729834,18.230000,18.124113,NaN
2443,2024-09-18,WSBF,15.920000,16.330000,15.700000,15.810000,37700.0,2.0,0.310502,15.754536,14.056464,62.162872,202.430213,35.166185,14.588333,14.209839,18.230000,18.124113,NaN


In [ ]:
train = pd.read_csv('/content/sample_data/train_data_20_40.csv', index_col='Unnamed: 0')

In [ ]:
train['sentiment'] = train.apply(merge_sentiment, axis=1)
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
0,2015-01-02,ALX,431.869995,440.459991,431.869995,248.700394,2100.0,4.0,0.000000,249.412768,247.208799,0.000000,-66.666667,100.000000,248.700394,248.700394,17.790001,0.000000,NaN
0,2015-01-02,ATLC,2.450000,2.680000,2.450000,2.600000,7000.0,4.0,0.000000,249.412768,247.208799,0.000000,-66.666667,100.000000,2.600000,2.600000,17.790001,0.000000,NaN
0,2015-01-02,BELFA,23.200001,23.200001,20.879999,19.717009,2500.0,4.0,0.000000,249.412768,247.208799,0.000000,-66.666667,100.000000,19.717009,19.717009,17.790001,0.000000,NaN
0,2015-01-02,CHMG,27.660000,27.850000,27.660000,20.917025,900.0,4.0,0.000000,249.412768,247.208799,0.000000,-66.666667,100.000000,20.917025,20.917025,17.790001,0.000000,NaN
0,2015-01-02,CZFS,50.307320,50.307320,50.307320,37.012135,104.0,4.0,0.000000,249.412768,247.208799,0.000000,-66.666667,100.000000,37.012135,37.012135,17.790001,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,2024-09-18,MBCN,27.510000,28.450001,26.670000,27.629999,33500.0,2.0,0.715305,29.109365,24.129709,56.878209,91.743556,24.607474,25.760080,24.936531,18.230000,2.107657,NaN
2443,2024-09-18,OBT,59.779999,59.799999,57.860001,59.060001,17400.0,2.0,0.790812,61.410507,53.404163,54.819410,96.257035,22.150659,56.215558,55.033078,18.230000,2.107657,NaN
2443,2024-09-18,OVLY,25.980000,26.980000,25.799999,26.240000,11700.0,2.0,0.097726,27.429340,24.396660,52.348711,70.690269,14.754692,25.817000,25.640770,18.230000,2.107657,NaN
2443,2024-09-18,PLPC,118.639999,120.500000,117.839996,119.213554,8800.0,2.0,-1.023339,122.577058,111.730493,49.013274,56.356884,1.077662,117.232652,120.780207,18.230000,2.107657,NaN


In [ ]:
train['sentiment'] = train['sentiment'].ffill().fillna(0)
train[train['sentiment']!=0]

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,sentiment
189,2015-10-02,BELFA,16.010000,16.110001,15.620000,14.296054,4300.0,4.0,-0.052147,14.845850,13.385065,46.189868,42.251723,6.125179,13.932186,15.166629,20.940001,0.000000,38.8368
189,2015-10-02,CHMG,27.389999,27.840000,27.389999,21.521248,500.0,4.0,0.328024,22.406334,19.794352,55.386651,72.769210,6.251577,20.783498,20.570144,20.940001,0.000000,38.8368
189,2015-10-02,CZFS,45.166077,45.166077,45.166077,34.136936,312.0,4.0,-0.056385,34.678295,33.292302,47.512503,-14.253888,3.210640,34.042883,34.555834,20.940001,0.000000,38.8368
189,2015-10-02,ESSA,12.950000,13.050000,12.950000,10.010116,6600.0,4.0,0.040786,10.147646,9.887438,52.521036,50.791311,29.586273,9.938232,9.915855,20.940001,0.000000,38.8368
189,2015-10-02,EVI,3.860000,3.990000,3.860000,3.592824,1100.0,4.0,-0.177575,4.784214,3.230967,41.817941,-104.319840,25.392421,3.999404,4.239077,20.940001,0.000000,38.8368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,2024-09-18,MBCN,27.510000,28.450001,26.670000,27.629999,33500.0,2.0,0.715305,29.109365,24.129709,56.878209,91.743556,24.607474,25.760080,24.936531,18.230000,2.107657,0.0329
2443,2024-09-18,OBT,59.779999,59.799999,57.860001,59.060001,17400.0,2.0,0.790812,61.410507,53.404163,54.819410,96.257035,22.150659,56.215558,55.033078,18.230000,2.107657,0.0329
2443,2024-09-18,OVLY,25.980000,26.980000,25.799999,26.240000,11700.0,2.0,0.097726,27.429340,24.396660,52.348711,70.690269,14.754692,25.817000,25.640770,18.230000,2.107657,0.0329
2443,2024-09-18,PLPC,118.639999,120.500000,117.839996,119.213554,8800.0,2.0,-1.023339,122.577058,111.730493,49.013274,56.356884,1.077662,117.232652,120.780207,18.230000,2.107657,0.0329


In [ ]:
train.to_csv('/content/sample_data/train_data_20_40.csv')